In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("data/clean.csv")
df

,nhs_num,lab_num,age,gender,location,location_other,size,mdm2,karyo_code,diagnosis,diagnosis_other,follow_up,mortality,cancer_mortality,comment,query,misc
0,7079315611,LH24-19172,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,-1.0,NaN,CF,NaN,NaN
1,4509208588,LH24-20991,74,f,o,breast,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1.0,NaN,CF,NaN,NaN
2,6342778353,LH24-18651,45,f,hn,right cheek parotid gland,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1.0,NaN,CF,Should parotid gland be in other?,NaN
3,6415674862,LH24-20355,23,f,lls,NaN,punch biopsy,False,mdm2 normal,other (free text),myxoid liposarcoma,NaN,-1.0,NaN,CF,Unsure whether diagnosis fits in option 'Pleom...,NaN
4,4520257115,LH24-19486,73,f,uls,NaN,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1.0,NaN,CF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,6049456305,BH09-3046,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865,4064416066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866,4343743969,LH09-10296,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,4382331137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mask = df["diagnosis"].str.contains("lipoma") | \
       df["diagnosis"].str.contains("liposarcoma") | \
       df["diagnosis_other"].str.contains("lipoma") | \
       df["diagnosis_other"].str.contains("liposarcoma")